In [1]:
######## Installations

%pip install pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [77]:
######## Imports

import pandas as pd
import os
import json
import csv
from datetime import datetime

In [71]:
######## Load the eval dataset
folder_path = './longmemeval_data/snippetized_data'
file_path = os.path.join(folder_path, 'longmemeval_oracle_snippetized.csv')
lme_dataset_df = pd.read_csv(file_path)
lme_dataset_df.head()

,question_id,question_type,multisession_index,session_index,message_index_within_session,message_index_across_sessions,session_date,message_role,message,previous_messages,num_previous_messages,message_has_answer
0,gpt4_2655b836,temporal-reasoning,0,0,0,0,2023-04-10 14:47:00,user,"""I'm thinking of getting a car wax and detaili...",[],0,False
1,gpt4_2655b836,temporal-reasoning,0,0,1,1,2023-04-10 14:47:00,assistant,"""Choosing the right detailer can make all the ...","[{""role"": ""user"", ""content"": ""I'm thinking of ...",1,False
2,gpt4_2655b836,temporal-reasoning,0,0,2,2,2023-04-10 14:47:00,user,"""I've been doing some research and found a loc...","[{""role"": ""user"", ""content"": ""I'm thinking of ...",2,True
3,gpt4_2655b836,temporal-reasoning,0,0,3,3,2023-04-10 14:47:00,assistant,"""That's great to hear that the dealership was ...","[{""role"": ""user"", ""content"": ""I'm thinking of ...",3,False
4,gpt4_2655b836,temporal-reasoning,0,0,4,4,2023-04-10 14:47:00,user,"""I'll definitely ask those questions when I vi...","[{""role"": ""user"", ""content"": ""I'm thinking of ...",4,False


In [129]:
######## Methods

MAX_NUM_PREVIOUS_MESSAGES = 5


def filter_for_zep_labelling(df):
    """
    Filters the dataset for the snippets we want to use for Zep labelling.
    """

    # Filter to only rows where question_type = single_session_user
    df = df[df['question_type'] == 'single-session-user']

    # Filter only where message_has_answer = True
    df = df[df['message_has_answer'] == True]

    # Filter to only rows where num_previous_messages = 5
    df = df[df['num_previous_messages'] == 5]

    # Limit to only 5 rows
    df = df.head(5)

    return df


def expand_previous_messages(df):
    """
    Expands the previous_messages column into separate columns.
    """
    # First parse the string into actual list of dicts
    df['previous_messages'] = df['previous_messages'].apply(json.loads)

    # Then create separate columns for each message
    for i in range(MAX_NUM_PREVIOUS_MESSAGES):
        df[f'previous_message_{i+1}'] = df['previous_messages'].apply(
            lambda x: x[i] if i < len(x) else None
        )

    # Drop the original previous_messages column if desired
    return df.drop('previous_messages', axis=1)



def make_messages_readable(df):
    """
    Makes the messages more readable.
    """
    for i in range(MAX_NUM_PREVIOUS_MESSAGES):
        df[f'previous_message_{i+1}'] = df[f'previous_message_{i+1}'].apply(
            lambda x: "|"*10 + f"  {x['role']}  " + "|"*10 + "\n\n" + f"{x['content']}" if x is not None else None
        )

    df['message'] = df.apply(
        lambda row: "|"*10 + f"  {row['message_role']}  " + "|"*10 + "\n\n" + f"{row['message']}" if row['message'] is not None else None,
        axis=1
    )
    return df


def order_columns(df):
    """
    Orders the columns in the way we want them.
    """
    df = df[['question_id', 'question_type', 'multisession_index', 'session_index', 'message_index_within_session', 'message_index_across_sessions', 'session_date',  'message_role', 'num_previous_messages', 'message_has_answer', 'previous_message_1', 'previous_message_2', 'previous_message_3', 'previous_message_4', 'previous_message_5', 'message']]
    return df



def insert_answer_columns(df, num_prompt_instructions):

    for prompt_instruction_index in range(num_prompt_instructions, 0, -1):

        for i in range(MAX_NUM_PREVIOUS_MESSAGES, 0, -1):
            # Insert after each previous message
            column_tag = f"({prompt_instruction_index}.{i})"
            df.insert(
                loc=df.columns.get_loc(f'previous_message_{i}') + 1,
                column=f'Answer to Prompt Instruction {prompt_instruction_index} {column_tag}',
                value=''
            )
            df.insert(
                loc=df.columns.get_loc(f'previous_message_{i}') + 2,
                column=f'Done?                {column_tag}',
                value=''
            )

        column_tag = f"({prompt_instruction_index}.{MAX_NUM_PREVIOUS_MESSAGES+1})"
        # Insert after each previous message
        df.insert(
            loc=df.columns.get_loc(f'message') + 1,
            column=f'Answer to Prompt Instruction {prompt_instruction_index} {column_tag}',
            value=''
        )
        df.insert(
            loc=df.columns.get_loc(f'message') + 2,
            column=f'Done?                {column_tag}',
            value=''
        )


def insert_default_answers_round1(df):
    """
    Inserts default answers for the first round of prompt instructions.
    """
    for i in range(MAX_NUM_PREVIOUS_MESSAGES, 0, -1):
        column_tag = f"(1.{i})"
        answer_col = f'Answer to Prompt Instruction 1 {column_tag}'
        msg_col = f'previous_message_{i}'
        
        # Set default value based on role from previous message
        df[answer_col] = df[msg_col].apply(
            lambda x: f"[${x['role']}$, ]" if x is not None else ""
        )

    # Handle the final message
    column_tag = f"(1.{MAX_NUM_PREVIOUS_MESSAGES+1})"
    answer_col = f'Answer to Prompt Instruction 1 {column_tag}'

    # Set default value based on role from current message
    df[answer_col] = df.apply(
        lambda row: f"[{row['message_role']},]",
        axis=1
    )

def insert_example_row(df, num_prompt_instructions):
    """
    Inserts an example row at the top of the dataframe with 'EXAMPLE' as values.
    """
    example_row = {col: "EXAMPLE" for col in df.columns}
    # for i in range(2):
    #     for j in range(num_prompt_instructions):
    #         example_row[f"Done? ({j+1}.{i+1})"] = "x"
    df.loc[-1] = example_row
    df.index = df.index + 1
    df.sort_index(inplace=True)
    return df



In [105]:
# Set pandas settings to display all columns and have max width of columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 120)


In [130]:
######## Filtering to only snippets/rows we want
lme_dataset_df_filtered = filter_for_zep_labelling(lme_dataset_df)
lme_dataset_df_filtered.head()

######## Expanding the previous_messages column
lme_dataset_df_filtered = expand_previous_messages(lme_dataset_df_filtered)
lme_dataset_df_filtered.head()

######## Order the columns in the way we want them
lme_dataset_df_filtered = order_columns(lme_dataset_df_filtered)
lme_dataset_df_filtered.head()

######## Insert empty answer columns
num_prompt_instructions = 1
insert_answer_columns(lme_dataset_df_filtered, num_prompt_instructions)
lme_dataset_df_filtered.head()

######## Insert default values for the answers
insert_default_answers_round1(lme_dataset_df_filtered)
lme_dataset_df_filtered.head()

######## Make the messages more readable
lme_dataset_df_filtered = make_messages_readable(lme_dataset_df_filtered)
lme_dataset_df_filtered.head(10)

######## Add example row to the top, with two examples filled in
insert_example_row(lme_dataset_df_filtered, num_prompt_instructions)
lme_dataset_df_filtered.head(10)

,question_id,question_type,multisession_index,session_index,message_index_within_session,message_index_across_sessions,session_date,message_role,num_previous_messages,message_has_answer,previous_message_1,Answer to Prompt Instruction 1 (1.1),Done? (1.1),previous_message_2,Answer to Prompt Instruction 1 (1.2),Done? (1.2),previous_message_3,Answer to Prompt Instruction 1 (1.3),Done? (1.3),previous_message_4,Answer to Prompt Instruction 1 (1.4),Done? (1.4),previous_message_5,Answer to Prompt Instruction 1 (1.5),Done? (1.5),message,Answer to Prompt Instruction 1 (1.6),Done? (1.6)
0,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE,EXAMPLE
6687,e47becba,single-session-user,286,0,5,5,2023-05-21 11:54:00,assistant,5,True,"|||||||||| user ||||||||||\n\nI'm trying to organize my life a bit better, can you recommend some task management ...","[$user$, ]",,|||||||||| assistant ||||||||||\n\nMaking the leap from a planner to a digital task management system! Congratulat...,"[$assistant$, ]",,|||||||||| user ||||||||||\n\nI think I'll try out Todoist and Trello. I've heard a lot of good things about them....,"[$user$, ]",,"|||||||||| assistant ||||||||||\n\nTodoist and Trello are both excellent choices for task management.\n\nNow, abou...","[$assistant$, ]",,"|||||||||| user ||||||||||\n\nI graduated with a degree in Business Administration, which has definitely helped me...","[$user$, ]",,"|||||||||| assistant ||||||||||\n\n""Congratulations on your degree in Business Administration! That's a great foun...","[assistant,]",
6702,118b2229,single-session-user,287,0,8,8,2023-05-23 18:02:00,user,5,True,"|||||||||| assistant ||||||||||\n\nGreat choice on Gone Girl!\n\nThe Nightingale is an excellent choice, and I'm h...","[$assistant$, ]",,|||||||||| user ||||||||||\n\nI've heard that audiobooks can be great for multitasking. Do you have any recommenda...,"[$user$, ]",,"|||||||||| assistant ||||||||||\n\nAudiobooks are perfect for multitasking, and note-taking apps can enhance your ...","[$assistant$, ]",,"|||||||||| user ||||||||||\n\nI've been using the Audible app to listen to my audiobooks, and it allows me to book...","[$user$, ]",,"|||||||||| assistant ||||||||||\n\nThe Audible app does offer some basic note-taking features, but it's great that...","[$assistant$, ]",,"|||||||||| user ||||||||||\n\n""I've been listening to audiobooks during my daily commute, which takes 45 minutes e...","[user,]",
6784,58ef2f1c,single-session-user,294,0,8,8,2023-04-02 00:23:00,user,5,True,"|||||||||| assistant ||||||||||\n\nWith your passion for animal welfare and children's health, I've curated a list...","[$assistant$, ]",,"|||||||||| user ||||||||||\n\nI think I'd like to learn more about the ""Strut Your Mutt"" event in September. Do yo...","[$user$, ]",,"|||||||||| assistant ||||||||||\n\n""Strut Your Mutt"" is a fantastic event that combines fun, exercise, and philant...","[$assistant$, ]",,"|||||||||| user ||||||||||\n\nCan you tell me more about the silent auction and raffles at the ""Strut Your Mutt"" e...","[$user$, ]",,"|||||||||| assistant ||||||||||\n\nThe silent auction and raffles at ""Strut Your Mutt"" are always a highlight of t...","[$assistant$, ]",,"|||||||||| user ||||||||||\n\n""I'm really looking forward to the silent auction and raffles at \""Strut Your Mutt\""...","[user,]",
6844,3b6f954b,single-session-user,299,0,8,8,2023-05-20 16:51:00,user,5,True,|||||||||| assistant ||||||||||\n\nThere are many amazing apps and tools out there that can help you stay organize...,"[$assistant$, ]",,"|||||||||| user ||||||||||\n\nI've been using Google Calendar to keep track of my classes and part-time job, but I...","[$user$, ]",,"|||||||||| assistant ||||||||||\n\nYou're already using Google Calendar, which is a great start! While Google Cale...","[$assistant$, ]",,"|||||||||| user 

In [131]:
# Save to csv
lme_dataset_df_filtered.to_csv("lme_dataset_filtered.csv", index=False)


# Archive

In [ ]:
######## Write a multisession to a txt file
question_id = "gpt4_93159ced_abs"
multi_session = lme_dataset_df[lme_dataset_df['question_id'] == question_id]["haystack_sessions"].iloc[0]


with open(f'{question_id}.txt', 'w') as f:


    for session in multi_session:
        f.write("New session " + "*"*200 + "\n")
        for message in session:
            
            f.write(f"{message}\n")
        f.write("\n")

In [61]:
######## Method to save all of the snippets (or only firsts/lasts) of the specified multi-sessions to a CSV file


def make_messages_readable(messages):
    if len(messages) == 0:
        return []
    if messages == [None]:
        return None
    result_string = ""
    for message in messages:
        if message is None:
            continue
        result_string += "|"*80 + f"  {message['role']}  " + "|"*80 + "\n\n" + f"{message['content']}\n\n"
    return result_string



def handle_message_readability(snippet, readableMessages, spreadOutPreviousMessages, max_num_previous_messages):
    if readableMessages:
        snippet['message'] = make_messages_readable([snippet['message']])
        if spreadOutPreviousMessages:
            for i in range(max_num_previous_messages):
                snippet[f"previous_message_{i+1}"] = make_messages_readable([snippet[f"previous_message_{i+1}"]])
            return snippet
        snippet['previous_messages'] = make_messages_readable(snippet['previous_messages'])
        return snippet
    
    snippet['message'] = json.dumps(snippet['message'])
    if spreadOutPreviousMessages:
        for i in range(max_num_previous_messages):
            snippet[f"previous_message_{i+1}"] = json.dumps(snippet[f"previous_message_{i+1}"])
        return snippet
    snippet['previous_messages'] = json.dumps(snippet['previous_messages'])
    return snippet

def save_multi_session_snippets_to_csv(question_ids, max_num_previous_messages=5, mode="all", readableMessages=False, spreadOutPreviousMessages=False, snippet_index=None):
    """
    Creates a csv where each row is a "snippet" from longmemeval. A snippet is a message and set of previous messages.

    mode:
        mode="all" --> saves all possible snippets for the specified question_ids. 
        mode="firsts_only" --> saves only the first snippet for each question_id.
        mode="lasts_only" --> saves only the last snippet for each question_id.
        mode="index_only" --> saves only the snippet at snippet_index for each question_id.

    readableMessages:
        readableMessages=True --> saves the messages in a readable format, intended for reading in Google sheets.
        readableMessages=False --> saves the messages in a json format.

    spreadOutPreviousMessages:
        spreadOutPreviousMessages=True --> spreads out the previous messages across multiple columns.
        spreadOutPreviousMessages=False --> saves all previous messages in a single column.
    
    """

    all_snippets = []
    indices = []
    for question_id in question_ids:
            
        ######## First, lets combine the sessions and dates into a list of dicts
        row = lme_dataset_df[lme_dataset_df['question_id'] == question_id]

        if row.empty:
            raise ValueError(f"No question found with ID: {question_id}")


        # Extract the haystack_sessions column value
        sessions = row['haystack_sessions'].iloc[0]

        # Get the haystack_dates column value
        session_dates = row['haystack_dates'].iloc[0]

        # Combine into list of dictionaries
        session_and_dates = [
            {
                "session": session,
                "date": datetime.strptime(date, "%Y/%m/%d (%a) %H:%M")
            } 
            for session, date in zip(sessions, session_dates)
        ]

        # Sort by date from earliest to latest
        session_and_dates.sort(key=lambda x: x["date"])


        all_snippets_this_session = []

        total_num_messages = sum([len(session_and_date["session"]) for session_and_date in session_and_dates])

        message_index_across_sessions = 0
        for session_index, session_and_date in enumerate(session_and_dates):
            for message_index_within_session, message in enumerate(session_and_date["session"]):
                
                num_previous_messages = min(max_num_previous_messages, message_index_across_sessions)
                previous_snippets = all_snippets_this_session[message_index_across_sessions-num_previous_messages:]
                previous_messages_only = [previous_snippet["message"] for previous_snippet in previous_snippets]

                snippet = {
                    "question_id": question_id,
                    "multisession_index": int(row.index[0]),
                    "session_index": session_index,
                    "message_index_within_session": message_index_within_session,
                    "session_date": session_and_date["date"],
                }

                if spreadOutPreviousMessages:
                    previous_messages_only_padded = previous_messages_only + [None] * (max_num_previous_messages - len(previous_messages_only))
                    for i, prev_msg in enumerate(previous_messages_only_padded):
                        snippet[f"previous_message_{i+1}"] = prev_msg
                    snippet["message"] = message
                else:
                    snippet["message"] = message
                    snippet["previous_messages"] = previous_messages_only

                all_snippets_this_session.append(snippet)
                message_index_across_sessions += 1

        if mode == "firsts_only":
            all_snippets_this_session = [all_snippets_this_session[0]]
        elif mode == "lasts_only":
            all_snippets_this_session = [all_snippets_this_session[-1]]
        elif mode == "index_only":
            all_snippets_this_session = [all_snippets_this_session[snippet_index]]

        all_snippets.extend(all_snippets_this_session)
        indices.append(int(row.index[0]))


    filename = "lme_samples_indices="
    indices.sort()  
    num_indices = len(indices)
    if num_indices < 4:
        for index in indices:
            filename += f"_{index}"
    else:
        filename += f"_{indices[0]}_etc_{indices[-1]}"

    if mode == "firsts_only":
        filename += "_firsts_only"
    elif mode == "lasts_only":
        filename += "_lasts_only"
    elif mode == "index_only":
        filename += f"_index_only={snippet_index}"
    if spreadOutPreviousMessages:
        filename += "_spreadOutPreviousMessages"
    if readableMessages:
        filename += "_readable"
    filename += ".csv"
    
    
    with open(filename, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=all_snippets[0].keys())
        writer.writeheader()
        for snippet in all_snippets:
            processed_snippet = handle_message_readability(snippet, readableMessages, spreadOutPreviousMessages, max_num_previous_messages)
            writer.writerow(processed_snippet)